In [1]:
%load_ext lab_black
%load_ext autoreload

In [2]:
import requests
import pandas as pd
import numpy as np
from typing import List, Dict
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform
import re
import os
import sys

sys.path.append("\campaign_mapping")
sys.path.append("\osm_extraktion")

from campaign_mapping import *
from osm_extraktion import *

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.min_rows", 4)

-- Firmennamen und ihre Adressen wird mit Hilfe die overpass API extraiert 


In [4]:
berlin = pd.read_parquet("OSM_Berlin.parquet")
berlin

,COMPANYNAME,AMENITY,OPERATOR,BRAND,OFFICE,SHOP,CITY,POSTALCODE,STREET,HSNR,LAT,LON
0,Berliner Volksbank,bank,None,Berliner Volksbank,None,None,Berlin,10713,Berliner Straße,42,52.486668,13.319723
1,Deutsche Post,post_office,Deutsche Post,Deutsche Post,None,None,Berlin,10717,Uhlandstraße,85,52.489074,13.322506
...,...,...,...,...,...,...,...,...,...,...,...,...
14643,Kleine Nordlichter,kindergarten,Kleine Nordlichter e.V.,None,None,None,Berlin,13467,Hermsdorfer Damm,192,NaN,NaN
14644,Zwergenschloß,kindergarten,Zwergenschloß gGmbH Kindertagesstätten,None,None,None,Berlin,13467,Olafstraße,16,NaN,NaN


In [32]:
berlin.info()
berlin.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14645 entries, 0 to 14644
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   COMPANYNAME  14645 non-null  object 
 1   AMENITY      5860 non-null   object 
 2   OPERATOR     4273 non-null   object 
 3   BRAND        2235 non-null   object 
 4   OFFICE       3140 non-null   object 
 5   SHOP         5679 non-null   object 
 6   CITY         14645 non-null  object 
 7   POSTALCODE   14645 non-null  object 
 8   STREET       14645 non-null  object 
 9   HSNR         14645 non-null  object 
 10  LAT          11672 non-null  float64
 11  LON          11672 non-null  float64
dtypes: float64(2), object(10)
memory usage: 1.3+ MB


In [34]:
berlin.columns

Index(['COMPANYNAME', 'AMENITY', 'OPERATOR', 'BRAND', 'OFFICE', 'SHOP', 'CITY',
       'POSTALCODE', 'STREET', 'HSNR', 'LAT', 'LON'],
      dtype='object')

In [6]:
berlin["OFFICE"].value_counts().reset_index().head(10)

,OFFICE,count
0,company,891
1,association,217
2,government,198
3,lawyer,184
4,insurance,174
5,it,155
6,diplomatic,151
7,architect,93
8,yes,91
9,ngo,89


In [7]:
# was sind unsere Top 3 in jeder Kategorie
v1 = berlin["AMENITY"].value_counts().reset_index().head(3)
v2 = berlin["SHOP"].value_counts().reset_index().head(3)
v3 = berlin["BRAND"].value_counts().reset_index().head(3)
v4 = berlin["OFFICE"].value_counts().reset_index().head(3)

top_3 = pd.concat([v1, v2, v3, v4], axis=1)
# display(v1, v2, v3, v4)
top_3

,AMENITY,count,SHOP,count,BRAND,count,OFFICE,count
0,kindergarten,1487,supermarket,1000,EDEKA,172,company,891
1,doctors,971,hairdresser,910,REWE,137,association,217
2,school,637,clothes,851,ALDI Nord,107,government,198


In [8]:
berlin_coordinates = berlin[berlin["LAT"].notna() & berlin["LON"].notna()]
berlin_campaign = berlin_coordinates[
    berlin_coordinates["OFFICE"].isin({"insurance", "educational_institution"})
]
berlin_campaign

,COMPANYNAME,AMENITY,OPERATOR,BRAND,OFFICE,SHOP,CITY,POSTALCODE,STREET,HSNR,LAT,LON
491,Babylonia,None,Babylonia e.V.,None,educational_institution,None,Berlin,10997,Cuvrystraße,23a,52.497176,13.442248
521,iu internationale Hochschule,None,None,None,educational_institution,None,Berlin,10247,Voigtstraße,2,52.515147,13.468030
...,...,...,...,...,...,...,...,...,...,...,...,...
11536,ANB Akademie für Notfallmanagement und Brandschutz GmbH,None,ANB Akademie für Notfallmanagement und Brandschutz GmbH,None,educational_institution,None,Berlin,12681,Boxberger Straße,3,52.545833,13.535427
11548,IBB Institut für berufliche Bildung,None,None,None,educational_institution,None,Berlin,10119,Torstraße,6,52.527365,13.414093


In [9]:
campaign_map(
    berlin_campaign,
    tag_name1="OFFICE",
    key1="insurance",
    tag_name2="OFFICE",
    key2="educational_institution",
)